In [ ]:
from sklearn.datasets import load_breast_cancer
bc = load_breast_cancer()

In [ ]:
bc.target_names

array(['malignant', 'benign'], dtype='<U9')

In [ ]:
import pandas as pd
bc_df = pd.DataFrame(bc.data, columns=bc.feature_names)
bc_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(bc_df, bc.target, test_size = 0.2, random_state = 31)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(455, 30)
(455,)
(114, 30)
(114,)


In [ ]:
!pip install aix360

     |████████████████████████████████| 58.3 MB 15 kB/s 
     |████████████████████████████████| 377 kB 65.4 MB/s 
     |████████████████████████████████| 109.3 MB 13 kB/s 
     |████████████████████████████████| 264 kB 66.7 MB/s 
     |████████████████████████████████| 109.7 MB 3.6 kB/s 
     |████████████████████████████████| 275 kB 77.7 MB/s 
     |████████████████████████████████| 50 kB 4.8 MB/s 
     |████████████████████████████████| 488 kB 67.5 MB/s 
     |████████████████████████████████| 3.1 MB 25.9 MB/s 
     |████████████████████████████████| 7.9 MB 38.8 MB/s 
     |████████████████████████████████| 15.7 MB 192 kB/s 
     |████████████████████████████████| 121 kB 78.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for lime: filename=lime-0.1.1.37-py3-none-any.whl size=284290 sha256=d9d93ca290caedaa287d324181d4593ce1fde37f5cc0ede2b331ad9b0aa79e47
  Stored in directory: /root/.

In [ ]:
from aix360.algorithms.rbm import FeatureBinarizer
fb = FeatureBinarizer(negations=True)
X_train_fb = fb.fit_transform(X_train)
X_test_fb = fb.transform(X_test)
X_train_fb['mean radius'][:8]

operation     <=                              ...      >                            
value     10.254 11.328 11.942 12.604 13.270  ... 13.270 14.142 15.058 17.026 19.324
468            0      0      0      0      0  ...      1      1      1      1      0
179            0      0      0      0      1  ...      0      0      0      0      0
114            1      1      1      1      1  ...      0      0      0      0      0
35             0      0      0      0      0  ...      1      1      1      0      0
88             0      0      0      1      1  ...      0      0      0      0      0
507            0      1      1      1      1  ...      0      0      0      0      0
213            0      0      0      0      0  ...      1      1      1      1      0
134            0      0      0      0      0  ...      1      1      1      1      0

[8 rows x 18 columns]

In [ ]:
from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG


In [ ]:
boolean_model = BooleanRuleCG(silent=True)
explainer = BRCGExplainer(boolean_model)
explainer.fit(X_train_fb, Y_train)

In [ ]:
Y_pred = explainer.predict(X_test_fb)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(f'Accuracy = {accuracy_score(Y_test, Y_pred)}')
print(f'Precision = {precision_score(Y_test, Y_pred)}')
print(f'Recall = {recall_score(Y_test, Y_pred)}')
print(f'F1 = {f1_score(Y_test, Y_pred)}')

Accuracy = 0.9298245614035088
Precision = 0.9538461538461539
Recall = 0.9253731343283582
F1 = 0.9393939393939394


In [ ]:
e = explainer.explain()
isCNF = 'Predict Y=0 if ANY of the following rules are satisfied, otherwise Y=1:'
notCNF = 'Predict Y=1 if ANY of the following rules are satisfied, otherwise Y=0:'
print(isCNF if e['isCNF'] else notCNF)
print()
for rule in e['rules']:
    print(f'  - {rule}')

Predict Y=1 if ANY of the following rules are satisfied, otherwise Y=0:

  - compactness error > 0.01 AND worst concavity <= 0.22 AND worst symmetry <= 0.28
  - mean texture <= 15.46 AND mean concavity <= 0.15 AND area error <= 54.16
  - fractal dimension error > 0.00 AND worst area <= 680.60 AND worst concave points <= 0.18
  - mean concave points <= 0.05 AND perimeter error <= 3.80 AND worst area <= 930.88 AND worst smoothness <= 0.16


In [ ]:

import pickle
# open a file, where you ant to store the data
file = open('breast_cancer_aix360_model.pkl', 'wb')

# dump information to that file
pickle.dump(explainer, file)